In [1]:
import sys
sys.path.append('../../30_data_tools/')
from pathlib import Path
from random import choices
from PIL import Image, ImageChops, ImageEnhance, ImageDraw
import numpy as np
import pandas as pd
import torch
from file_interaction import download_blob, get_related_filepath, open_img
from tqdm.auto import tqdm
import sqlite3
import re
import cv2
# import YOLO model
from ultralytics import YOLO
from io import BytesIO
from torchvision.transforms.functional import pil_to_tensor

In [2]:
from helper import load_dotenv
from get_labelstudio_data import get_results_of_project

In [87]:
Image.MAX_IMAGE_PIXELS = None

In [3]:
TILE_SIZE = 224
MIN_MIDTONE_SHARE = 0.05
MARGIN = 0.03
MAX_BATCH_SIZE = 64

In [4]:
model_name = '2024-04-11_resnet50_003'

In [5]:
dotenv = load_dotenv()
con = sqlite3.connect( dotenv['DB_PATH'] )

In [6]:
exclude_jobs = [
    '23-10-03_Testformen',
    '23-10-18_farbe',
    '23-10-17_blur',
    '23-10-19_farbe'
]

include_jobs = [
    '24-03-05-01_randomTrainPages'
]

In [7]:
results = [
    r for r in get_results_of_project(3)
    if r['labels'][0] not in ['potential_moire'] and True not in [r['img_name'].startswith(ej) for ej in exclude_jobs]
]

In [8]:
pages = pd.read_sql(
    f'''
        SELECT rf.* FROM related_file rf 
        LEFT JOIN (SELECT job, pdf_filename, 1 AS used_as_base FROM generic_image gi ) gi
        ON rf.job=gi.job AND rf.pdf_filename = gi.pdf_filename
        WHERE variant_name = 'ps2400dpi150lpi' AND "type" = '4c_{ dotenv["LOFI_DPI"] }' AND gi.used_as_base IS NULL
    ''',
    con
)

In [9]:
bytesStream = download_blob( f'models/{ model_name }.pth' )
model = torch.load( BytesIO(bytesStream.getvalue()), map_location=torch.device('cpu') )

# Funktionen

In [21]:
def cut_img_into_tiles( img ):
    tiles = []    
    left = 0
    top = 0
    
    while left < img.size[0] or top < img.size[1]:
        tile = img.crop((
            left,top,
            left+TILE_SIZE,top+TILE_SIZE
        ))
        tiles.append( ((left,top),tile) )
    
        if left < img.size[0]:
            left += round(TILE_SIZE / 2)
        elif top < img.size[1]:
            left = 0
            top += TILE_SIZE
        else:
            break

    return tiles

In [211]:
def preclassifier( tiles ):
    tiles_out = []
    
    for pos,tile in tiles:
        is_relevant = False
        np_tile = 1 - np.array(tile) / 255
        
        for i in range( np_tile.shape[2] ):
            sep = np_tile[:,:,i]
            midtone_share = sep[(sep > MARGIN) & (sep < (1 - MARGIN))].shape[0] / (sep.shape[0] * sep.shape[1])
        
            if midtone_share > MIN_MIDTONE_SHARE:
                is_relevant = True
                break

        if is_relevant:
            tiles_out.append((pos,tile))

    return tiles_out

In [231]:
def classifier( tiles ):
    batch_start = 0
    predictions = []
    
    with tqdm(total=len(tiles)) as pbar:
        while batch_start < len(tiles):
            tile_selection = tiles[batch_start:batch_start+MAX_BATCH_SIZE]
            batch = torch.zeros([len(tile_selection), 3, TILE_SIZE, TILE_SIZE], dtype=torch.float32)
        
            for i in range(len(tile_selection)):
                batch[i,:,:,:] = 1 - pil_to_tensor(Image.fromarray(np.array(tile_selection[i][1])[:,:,3]).convert('RGB')) / 255
            
            model.eval()

            with torch.no_grad():
                model_predictions = model(batch)
                predictions += [(*tile_selection[i],model_predictions[i],int(torch.argmax(model_predictions[i]))) for i in range(len(model_predictions))]
    
            batch_start += MAX_BATCH_SIZE
            pbar.update(len(tile_selection))

    return predictions

In [227]:
def get_out_map( img, relevant_tiles ):
    out_img = np.zeros((img.size[1], img.size[0], 4))
    colors = [
        (0,0,255),
        (255,0,0)
    ]
    
    for pos,tile,pred,label in relevant_tiles:
        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            0
        ] += 1

        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            1
        ] += float(pred[1])

        out_img[
            pos[1]:pos[1]+TILE_SIZE,
            pos[0]:pos[0]+TILE_SIZE,
            2
        ] += float(pred[0])

    out_img /= 2

    level_1_img = Image.fromarray((out_img[:,:,0] * 255).astype('uint8'))
    level_2_img = Image.fromarray((out_img[:,:,1] * 255).astype('uint8'))
    level_3_img = Image.fromarray((out_img[:,:,2] * 255).astype('uint8'))
    level_4_img = out_img[:,:,2] * 255
    level_4_img[(out_img[:,:,2] < out_img[:,:,1]) | (out_img[:,:,2] < 0.5)] = 0
    level_4_img = Image.fromarray(level_4_img.astype('uint8'))
    
    return level_1_img, level_2_img, level_3_img, level_4_img

In [47]:
def create_result_map( img_size, tiles ):
    result_map = np.zeros((img_size[1],img_size[0])).astype('float32')
    
    for pos,tile,pred,label in tiles:
        if pred[1] < pred[0]:
            result_map[
                pos[1]:pos[1]+TILE_SIZE,
                pos[0]:pos[0]+TILE_SIZE,
            ] += float(pred[0]) / 2
            # / 2 weil max. zwei tiles die Kachel bestimmen

    return result_map

In [15]:
def get_moires_of_page( row, results ):
    relevant_moires = []
    
    for r in results:
        if re.match(f'^{ row.job }\..+?\.{ row.pdf_filename }\..+', r['img_name']):
            dpi = int( re.match(r'.+\.4c_(\d+)\.jpg', r['img_name']).groups()[0] )
            out_box = [r['value']['x'],r['value']['y'],r['value']['width'],r['value']['height']]
            
            # box umrechnen
            if target_dpi != dpi:
                out_box = [round(val * (target_dpi / dpi)) for val in out_box]
    
            relevant_moires.append(out_box)

    return relevant_moires

In [16]:
def get_intersection_over_union( box_a, box_b ):
    intersection_box = [
        max(box_a[0],box_b[0]),
        max(box_a[1],box_b[1]),
        min(box_a[0]+box_a[2],box_b[0]+box_b[2]),
        min(box_a[1]+box_a[3],box_b[1]+box_b[3]),
    ]

    if intersection_box[2] - intersection_box[0] < 0 or intersection_box[3] - intersection_box[1] < 0:
        return 0

    intersection = (intersection_box[2] - intersection_box[0]) * (intersection_box[3] - intersection_box[1])
    
    union_box = [
        min(box_a[0],box_b[0]),
        min(box_a[1],box_b[1]),
        max(box_a[0]+box_a[2],box_b[0]+box_b[2]),
        max(box_a[1]+box_a[3],box_b[1]+box_b[3]),
    ]
    union = (union_box[2] - union_box[0]) * (union_box[3] - union_box[1])

    return intersection / union

In [17]:
def get_result_boxes( result_map, threshold=0.5 ):
    thresh = np.zeros(result_map.shape).astype('uint8')
    thresh[result_map > threshold] = 255

    (numLabels, labels, stats, centroids) = cv2.connectedComponentsWithStats(
    	thresh, 4, cv2.CV_32S
    )

    return [
        [b[0],b[1],b[2],b[3]]
        for b in stats[1:]
    ]

In [18]:
def draw_bounding_boxes( img, moire_boxes, predicted_boxes ):
    colors = {
        "target" : "green",
        "predicted" : "red"
    }    

    img_out = img.copy().convert('RGB')
    draw = ImageDraw.Draw(img_out) 

    for b in moire_boxes:
        draw.rectangle([b[0],b[1],b[0]+b[2],b[1]+b[3]], outline=colors['target'], width=10) 

    for b in predicted_boxes:
        draw.rectangle([b[0],b[1],b[0]+b[2],b[1]+b[3]], outline=colors['predicted'], width=10) 

    
    return img_out

In [19]:
target_dpi = dotenv['TRAIN_DATA_DPI']

In [20]:
0 / 0

ZeroDivisionError: division by zero

In [241]:
samples = pages.sample(n=10)

In [243]:
for i in tqdm(range(samples.shape[0])):
    sample = samples.iloc[i]
    
    img_path = get_related_filepath(
        sample.job,
        'ps2400dpi150lpi',
        f'{ sample.pdf_filename }.4c_{ dotenv["LOFI_DPI"] }.jpg'
    )
    img = open_img(img_path)
    img = img.resize((
        round(img.size[0] * (target_dpi / dotenv["LOFI_DPI"])),
        round(img.size[1] * (target_dpi / dotenv["LOFI_DPI"]))
    ))
    tiles = cut_img_into_tiles( img )
    tiles_preclassified = preclassifier( tiles )
    predictions = classifier( tiles_preclassified )
    
    result_map = create_result_map( img.size, predictions )
    relevant_moires = get_moires_of_page( sample, results )
    
    predicted_label = int(result_map[result_map > 0.5].shape[0] > 0)

    # blend img erzeugen
    TARGET_OUT_HEIGHT = 1000
    colors=['red','green','blue','orange']
    l_images = get_out_map( img, predictions )
    blended = Image.new(mode="RGB", size=(img.size[0] * len(l_images), img.size[1]))

    for i in range(len(l_images)):
        l_img = ImageEnhance.Brightness(l_images[i]).enhance(0.5)
        overlay = Image.new('RGB', l_img.size, color=colors[i])
        l_rgb = img.convert('RGB')
        l_rgb.paste(
            overlay,
            (0,0),
            mask=l_img
        )

        blended.paste(
            l_rgb,
            (img.size[0] * i,0)
        )

    #blended = blended.resize((
    #    round(TARGET_OUT_HEIGHT / blended.size[1] * blended.size[0]),
    #    TARGET_OUT_HEIGHT
    #))
    
    blended.save( dotenv['TEMP_PROCESSING_DIR'] / 'blended_page_results' / f"separated.{ sample.job }.{ sample.pdf_filename }.{ predicted_label }.jpg", progressive=True )

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/146 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

...

  0%|          | 0/121 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/329 [00:00<?, ?it/s]

  0%|          | 0/340 [00:00<?, ?it/s]